# Pandas API
The purpose of this notebook is to provide a demonstration of the capabilities of the pandas like API for PyKX Table objects.

To follow along please download this notebook using the following <a href="./Pandas_API.ipynb" download>link</a>.

This demonstration will outline the following

1. [Constructing Tables](#Constructing-Tables)
2. [Metadata](#Metadata)
3. [Querying and Data Interrogation](#Querying-and-Data-Interrogation)
4. [Data Joins/Merging](#Data-Joins/Merging)
5. [Analytic Functionality](#Analytic-functionality)
6. [Data Preprocessing](#Data-Preprocessing)

In [ ]:
import os
os.environ['IGNORE_QHOME'] = '1' # Ignore symlinking PyKX q libraries to QHOME 
os.environ['PYKX_Q_LOADED_MARKER'] = '' # Only used here for running Notebook under mkdocs-jupyter during document generation.


In [ ]:
import pykx as kx
import numpy as np
import pandas as pd
kx.q.system.console_size = [10, 80]

## Constructing Tables

### Table

Create a table from a list of rows or by converting a Python dictionary object

**Parameters:**

| Name    | Type               | Description                                                            | Default  |
| :-----: | :----------------: | :--------------------------------------------------------------------- | :------: |
| x       | Union[list, array] | An array like object containing the contents of each row of the table. | None     |
| data    | dict               | A dictionary to be converted into a Table object.                      | None     |
| columns | list[str]          | A list of column names to use when constructing from an array of rows. | None     |

**Returns:**

| Type  | Description                         |
| :---: | :---------------------------------- |
| Table | The newly constructed table object. |

**Examples:**

Create a table from a dictionary object.

In [ ]:
kx.Table(data={'x': list(range(10)), 'y': [10 - x for x in range(10)]})

Create a Table from an array like object.

In [ ]:
kx.Table([[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]])

Create a Table from an array like object and provide names for the columns to use.

In [ ]:
kx.Table([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]], columns=['x', 'y', 'z'])

### Keyed Table

Create a keyed table from a list of rows or by converting a Python dictionary object

**Parameters:**

| Name    | Type               | Description                                                            | Default  |
| :-----: | :----------------: | :--------------------------------------------------------------------- | :------: |
| x       | Union[list, array] | An array like object containing the contents of each row of the table. | None     |
| data    | dict               | A dictionary to be converted into a Table object.                      | None     |
| columns | list[str]          | A list of column names to use when constructing from an array of rows. | None     |
| index   | list[Any]          | An array like object to use as the index column of the table.          | None     |

**Returns:**

| Type       | Description                               |
| :--------: | :---------------------------------------- |
| KeyedTable | The newly constructed keyed table object. |

**Examples:**

Create a keyed table from a dictionary object.

In [ ]:
kx.KeyedTable(data={'x': list(range(10)), 'y': list(10 - x for x in range(10))})

Create a keyed table from a list of rows.

In [ ]:
kx.KeyedTable([[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]])

Create a keyed table from a list of rows and provide names for the resulting columns.

In [ ]:
kx.KeyedTable([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]], columns=['x', 'y', 'z'])

Create a keyed table with a specified index column.

In [ ]:
kx.KeyedTable(data={'x': list(range(10)), 'y': list(10 - x for x in range(10))}, index=[2 * x for x in range(10)])

## Metadata

In [ ]:
kx.q('N: 1000')
tab = kx.q('([] x: til N; y: N?`AAPL`GOOG`MSFT; z: N?500f; w: N?1000; v: N?(0N 0 50 100 200 250))')
tab

### Table.columns

Get the name of each column in the table

In [ ]:
tab.columns

### Table.dtypes

Get the pandas dtype of each column

In [ ]:
tab.dtypes

### Table.empty

Returns True if the table is empty otherwise returns False.

In [ ]:
tab.empty

### Table.ndim

Get the nuber of columns within the table.

In [ ]:
tab.ndim

### Table.shape

Get the shape of the table as a tuple (number of rows, number of columns).

In [ ]:
tab.shape

### Table.size

Get the number of values in the table (rows * cols).

In [ ]:
tab.size

## Querying and Data Interrogation

In [ ]:
# The examples in this section will use this example table filled with random data
kx.q('N: 1000')
tab = kx.q('([] x: til N; y: N?`AAPL`GOOG`MSFT; z: N?500f; w: N?1000; v: N?(0N 0 50 100 200 250))')
tab

### Table.all()

```
Table.all(axis=0, bool_only=False, skipna=True)
```

Returns whether or not all values across the given axis have a `truthy` value.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate `all` across 0 is columns, 1 is rows.                      | 0       |
| bool_only    | bool | Only use columns of the table that are boolean types.                            | False   |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represents the column name / row number and the values are the result of calling `all` on that column / row. |

In [ ]:
tab.all()

### Table.any()

```
Table.any(axis=0, bool_only=False, skipna=True)
```

Returns whether or not any values across the given axis have a `truthy` value.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate `any` across 0 is columns, 1 is rows.                      | 0       |
| bool_only    | bool | Only use columns of the table that are boolean types.                            | False   |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represents the column name / row number and the values are the result of calling `any` on that column / row. |

In [ ]:
tab.any()

### Table.at[]

```
Table.at[row, col]
```

Access a single value for a row / column pair.

Similar to `loc[]`, in that both provide label-based lookups. Use `at` if you only need to get or set a single value.

The `at` property can be used for both assignment and retrieval of values at a given row and column.

**Examples:**

Get the value of the `z` column in the 997th row.

In [ ]:
tab.at[997, 'z']

Reassign the value of the `z` column in the 997th row to `3.14159`.

In [ ]:
tab.at[997, 'z'] = 3.14159
tab.at[997, 'z']

### Table.get()

```
Table.get(key, default=None)
```

Get a column or columns from a table by key, if the key does not exist return the default value.

**Parameters:**

| Name    | Type                   | Description                                             | Default    |
| :-----: | :--------------------: | :------------------------------------------------------ | :--------: |
| key     | Union[str, list[str]]  | The column name or list of names to get from the table. | _required_ |
| default | int                    | The default value if the key is not found.              | None       |

**Returns:**

| Type              | Description                                                          |
| :---------------: | :------------------------------------------------------------------- |
| Union[Table, Any] | A table containing only the columns requested or the default value. |

**Examples:**

Get the `y` column from the table.

In [ ]:
tab.get('y')

Get the `y` and `z` columns from the table.

In [ ]:
tab.get(['y', 'z'])

Attempt to get the `q` column from the table and receive none as that column does not exist.

In [ ]:
print(tab.get('q'))

Attempt to get the `q` column from the table and receive the default value `not found` as that column does not exist.

In [ ]:
tab.get('q', 'not found')

### Table.head()

```
Table.head(n=5)
```

Get the first n rows from a table.

**Parameters:**

| Name | Type | Description                   | Default |
| :--: | :--: | :---------------------------- | :-----: |
| n    | int  | The number of rows to return. | 5       |

**Returns:**

| Type  | Description                      |
| :---: | :------------------------------- |
| Table | The first `n` rows of the table. |

**Examples:**

Return the first 5 rows of the table.

In [ ]:
tab.head()

Return the first 10 rows of the table.

In [ ]:
tab.head(10)

### Table.isna()

```
Table.isna()
```

Detects null values on a Table object.

**Parameters:**


**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Table         | A Table with the same shape as the original but containing boolean values. 1b represents a null value was on its place and 0b represents the opposite. |

In [ ]:
tab.isna()

### Table.isnull()

```
Table.isnull()
```

Alias of Table.isna().

Detects null values on a Table object.

**Parameters:**


**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Table         | A Table with the same shape as the original but containing boolean values. 1b represents a null value was on its place and 0b represents the opposite. |

In [ ]:
tab.isnull()

### Table.notna()

```
Table.notna()
```

Boolean inverse of Table.isna().

Detects non-null values on a Table object.

**Parameters:**


**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Table         | A Table with the same shape as the original but containing boolean values. 1b represents a non null value was on its place and 0b represents the opposite. |

In [ ]:
tab.notna()

### Table.notnull()

```
Table.notna()
```

Boolean inverse of Table.isnull(). Alias of Table.isna()

Detects non-null values on a Table object.

**Parameters:**


**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Table         | A Table with the same shape as the original but containing boolean values. 1b represents a non null value was on its place and 0b represents the opposite. |

In [ ]:
tab.notnull()

### Table.iloc[]

```
Table.iloc[:, :]
```

Purely integer-location based indexing for selection by position.

`iloc` is primarily integer position based (from 0 to length-1 of the axis), but may also be used with a BooleanVector.

Allowed inputs are:
- An integer, e.g. 5.
- A list or array of integers, e.g. [4, 3, 0].
- A slice object with ints, e.g. 1:7.
- A BooleanVector.
- A callable function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above). This is useful in method chains, when you don’t have a reference to the calling object, but would like to base your selection on some value.
- A tuple of row and column indexes. The tuple elements consist of one of the above inputs, e.g. (0, 1).

**Returns:**

| Type  | Description                                            |
| :---: | :----------------------------------------------------- |
| Table | A table containing only the columns / rows requested. |

**Examples:**

Get the second row from a table.

In [ ]:
tab.iloc[1]

Get the first 5 rows from a table.

In [ ]:
tab.iloc[:5]

Get all rows of the table where the `y` column is equal to `AAPL`.

In [ ]:
tab.iloc[tab['y'] == 'AAPL']

Get all rows of the table where the `y` column is equal to `AAPL`, and only return the `y`, `z` and `w` columns.

In [ ]:
tab.iloc[tab['y'] == 'AAPL', ['y', 'z', 'w']]

Replace all null values in the column `v` with the value `-100`.

In [ ]:
tab.iloc[tab['v'] == kx.q('0N'), 'v'] = -100
tab

### Table.loc[]

```
Table.loc[:, :]
```

Access a group of rows and columns by label or by BooleanVector.

`loc` is a label based form of indexing, but may also be used with a boolean array.

Allowed inputs are:

- A single label, e.g. 5 or 'a', (note that 5 is interpreted as a label of the index, and never as an integer position along the index)
- A list or array of labels, e.g. ['a', 'b', 'c']
- A slice object with labels, e.g. 'a':'f'
    - **Warning contrary to usual python slices, both the start and the stop are included**
- A BooleanVector of the same length as the axis being sliced
- An alignable BooleanVector. The index of the key will be aligned before masking
- An alignable Index. The Index of the returned selection will be the input
- A callable function with one argument (the calling Table like object) and that returns valid output for indexing (e.g. one of the above)

Note:
When the Pandas API is enabled, using `[]` to index into a table will use `Table.loc[]`

**Returns:**

| Type  | Description                                            |
| :---: | :----------------------------------------------------- |
| Table | A table containing only the columns / rows requested. |

**Examples:**

Get every row in the `y` column.

In [ ]:
tab[:, 'y']

Get all rows of the table where the value in the `z` column is greater than `250.0`

In [ ]:
tab[tab['z'] > 250.0]

Replace all null values in the column `v` with the value `-100`.

In [ ]:
tab.loc[tab['v'] == kx.q('0N'), 'v'] = -100
tab

Replace all locations in column `v` where the value is `-100` with a null.

In [ ]:
tab[tab['v'] == -100, 'v'] = kx.q('0N')
tab

Usage of the `loc` functionality under the hood additionally allows users to set columns within a table for single or multiple columns. Data passed for this can be q/Python.

In [ ]:
tab['new_col'] = kx.q('1000?1f')

In [ ]:
tab[['new_col1', 'new_col2']] = [20, kx.q('1000?0Ng')]

### Table.sample()

```
Table.sample(n, frac, replace, weights, random_state, axis, ignore_index)
```

Sample random data from the table.

**Parameters:**

| Name         | Type  | Description                                                        | Default |
| :----------: | :---: | :----------------------------------------------------------------- | :-----: |
| n            | int   | Number of rows to return. Cannot be used with `frac`. Default is 1 if `frac` is None. | None |
| frac         | float | Fraction of the rows to return. Cannot be used with `n`.           | None    |
| replace      | bool  | Whether or not it should be possible to sample the same row twice. | False   |
| weights      | None  | Not yet implemented.                                               | None    |
| random_state | None  | Not yet implemented.                                               | None    |
| axis         | None  | Not yet implemented.                                               | None    |
| ignore_index | bool  | Not yet implemented.                                               | False   |

**Returns:**

| Type  | Description                                                        |
| :---: | :----------------------------------------------------------------- |
| Table | A table with the given column(s) renamed.                          |

In [ ]:
# The examples in this section will use this example table filled with random data
kx.q('N: 1000')
tab = kx.q('([] x: til N; y: N?`AAPL`GOOG`MSFT; z: N?500f; w: N?1000; v: N?(0N 0 50 100 200 250))')
tab.head()

**Examples:**

Sample 10 Rows.

In [ ]:
tab.sample(n=10)

Sample 10%  of the rows.

In [ ]:
tab.sample(frac=0.1)

Sample 10%  of the rows and allow the same row to be sampled twice.

In [ ]:
tab.sample(frac=0.1, replace=True)

### Table.select_dtypes()

```
Table.select_dtypes(include=None, exclude=None)
```

Return a subset of the DataFrame’s columns based on the column dtypes.

Allowed inputs for `include`/`exclude` are:
- A single dtype or string.
- A list of dtypes or strings.
- Inputs given for `include` and `exclude` cannot overlap.

The dtype `kx.CharVector` will return an error. Use `kx.CharAtom` for a column of single chars.
Both `kx.*Atom` and `kx.*Vector` will be taken to mean a column containing a single item per row of type `*`.  `kx.List` will include/exclude any columns containing mixed list data (including string columns).
    
**Parameters:**

| Name    | Type             | Description                                      | Default |
| :-----: | :--------------: | :----------------------------------------------: | :-----: |
| include | Union[List, str] | A selection of dtypes or strings to be included. | None    |
| exclude | Union[List, str] | A selection of dtypes or strings to be excluded. | None    |
    
At least one of these parameters must be supplied.
    
**Returns:**

| Type      | Description                                                                                      |
| :-------: | :----------------------------------------------------------------------------------------------: |
| Dataframe | The subset of the frame including the dtypes in `include` and excluding the dtypes in `exclude`. |

**Examples:**

The examples in the section will use the example table.

In [ ]:
df = kx.q('([] c1:`a`b`c; c2:1 2 3h; c3:1 2 3j; c4:1 2 3i)')

Exclude columns containing symbols

In [ ]:
df.select_dtypes(exclude = kx.SymbolVector)

Include a list of column types

In [ ]:
df.select_dtypes(include = [kx.ShortVector, kx.LongVector])

### Table.tail()

```
Table.tail(n=5)
```

Get the last n rows from a table.

**Parameters:**

| Name | Type | Description                   | Default |
| :--: | :--: | :---------------------------- | :-----: |
| n    | int  | The number of rows to return. | 5       |

**Returns:**

| Type  | Description                      |
| :---: | :------------------------------- |
| Table | The last `n` rows of the table.  |

**Examples:**

Return the last 5 rows of the table.

In [ ]:
tab.tail()

Return the last 10 rows of the table.

In [ ]:
tab.tail(10)

## Data Joins/Merging

### Table.merge()

```
Table.merge(
    right,
    how='inner',
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=('_x', '_y'),
    copy=True,
    validate=None,
    q_join=False
)
```

Merge Table or KeyedTable objects with a database-style join.

The join is done on columns or keys. If joining columns on columns, the Table key will be ignored. Otherwise if joining keys on keys or keys on a column or columns, the index will be passed on. When performing a cross merge, no column specifications to merge on are allowed.

**Parameters:**

| Name        | Type | Description                                                                       | Default |
| :---------: | :--: | :-------------------------------------------------------------------------------- | :-----: |
| right       | Union[Table/KeyedTable] | The object to merge with.                                      | _required_ |
| how         | str  | The type of join to be used. One of {‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}. | ‘inner’ |
| on          | str  | The column name to join on.                                                       | None    |
| left_on     | str  | The column name in the left table to join on.                                     | None    |
| right_on    | str  | The column name in the right table to join on.                                    | None    |
| left_index  | bool | Use the index of the left Table.                                                  | False   |
| right_index | bool | Use the index of the right Table.                                                 | False   |
| sort        | bool | Sort the join keys of the resulting table.                                        | False   |
| suffixes    | Tuple(str, str) | The number of rows to return.                                          | ('\_x', '\_y') |
| copy        | bool | If False avoid copies and modify the input table.                                 | None    |
| validate    | str  | If specified checks if merge matches specified type.<br> - “one_to_one” or “1:1”: check if merge keys are unique in both left and right datasets.<br> - “one_to_many” or “1:m”: check if merge keys are unique in left dataset.<br> - “many_to_one” or “m:1”: check if merge keys are unique in right dataset.<br> - “many_to_many” or “m:m”: allowed, but does not result in checks.<br> | None    |
| q_join      | bool | If True perform native q joins instead of the pandas SQL like joins. More documentation around these joins can be found [here.](https://code.kx.com/q/basics/joins/) | False   |

**Returns:**

| Type               | Description                                                         |
| :----------------: | :------------------------------------------------------------------ |
| Table / KeyedTable | The resulting table-like object after the join has been preformed. |

**Examples:**

Merge tab1 and tab2 on the lkey and rkey columns. The value columns have the default suffixes, \_x and \_y, appended.

In [ ]:
tab1 = kx.Table(data={'lkey': ['foo', 'bar', 'baz', 'foo'], 'value': [1, 2, 3, 5]})
tab2 = kx.Table(data={'rkey': ['foo', 'bar', 'baz', 'foo'], 'value': [5, 6, 7, 8]})
tab1.merge(tab2, left_on='lkey', right_on='rkey')

Merge tab1 and tab2 on the lkey and rkey columns using a native q inner join. The value columns have the default suffixes, \_x and \_y, appended.

In [ ]:
tab1.merge(tab2, left_on='lkey', right_on='rkey', q_join=True)

Merge tab1 and tab2 with specified left and right suffixes appended to any overlapping columns.

In [ ]:
tab1.merge(tab2, left_on='lkey', right_on='rkey', suffixes=('_left', '_right'))

Merge tab1 and tab2 but raise an exception if the Tables have any overlapping columns.

In [ ]:
try:
    tab1.merge(tab2, left_on='lkey', right_on='rkey', suffixes=(False, False))
except BaseException as e:
    print(f'Caught Error: {e}')

In [ ]:
tab1 = kx.Table(data={'a': ['foo', 'bar'], 'b': [1, 2]})
tab2 = kx.Table(data={'a': ['foo', 'baz'], 'c': [3, 4]})

Merge tab1 and tab2 on the `a` column using an inner join.

In [ ]:
tab1.merge(tab2, how='inner', on='a')

Merge tab1 and tab2 on the `a` column using a left join.

In [ ]:
tab1.merge(tab2, how='left', on='a')

Merge tab1 and tab2 using a cross join.

In [ ]:
tab1 = kx.Table(data={'left': ['foo', 'bar']})
tab2 = kx.Table(data={'right': [7, 8]})
tab1.merge(tab2, how='cross')

### Table.merge_asof()

```
Table.merge_asof(
    right,
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    by=None,
    left_by=None,
    right_by=None,
    suffixes=('_x', '_y'),
    tolerance=None,
    allow_exact_matches=True,
    direction='backward'

)
```

Merge Table or KeyedTable objects with a database-style join.

The join is done on columns or keys. If joining columns on columns, the Table key will be ignored. Otherwise if joining keys on keys or keys on a column or columns, the index will be passed on. When performing a cross merge, no column specifications to merge on are allowed.

**Parameters:**

| Name        | Type | Description                                                                       | Default |
| :---------: | :--: | :-------------------------------------------------------------------------------- | :-----: |
| right       | Union[Table/KeyedTable] | The object to merge with.                                      | _required_ |
| how         | str  | The type of join to be used. One of {‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}. | ‘inner’ |
| on          | str  | The column name to join on.                                                       | None    |
| left_on     | str  | The column name in the left table to join on.                                     | None    |
| right_on    | str  | The column name in the right table to join on.                                    | None    |
| left_index  | bool | Use the index of the left Table.                                                  | False   |
| right_index | bool | Use the index of the right Table.                                                 | False   |
| by          | str  | Not yet implemented.                                                              | None    |
| left_by     | str  | Field names to match on in the left table.                                        | None    |
| right_by    | str  | Field names to match on in the right table.                                       | None    |
| suffixes    | Tuple(str, str) | The number of rows to return.                                          | ('\_x', '\_y') |
| tolerance   | Any   | Not yet implemented.                                                             | None    |
| allow_exact_matches | bool | Not yet implemented.                                                      | True    |
| direction   | str   | Not yet implemented.                                                             | 'backward' |


**Returns:**

| Type               | Description                                                         |
| :----------------: | :------------------------------------------------------------------ |
| Table / KeyedTable | The resulting table like object after the join has been preformed. |

**Examples:**

Perform a simple asof join on two tables.

In [ ]:
left  = kx.Table(data={"a": [1, 5, 10], "left_val": ["a", "b", "c"]})
right = kx.Table(data={"a": [1, 2, 3, 6, 7], "right_val": [1, 2, 3, 6, 7]})
left

In [ ]:
right

In [ ]:
left.merge_asof(right)

Perform a asof join on two tables but first merge them on the by column.

In [ ]:
trades = kx.Table(data={
    "time": [
        pd.Timestamp("2016-05-25 13:30:00.023"),
        pd.Timestamp("2016-05-25 13:30:00.023"),
        pd.Timestamp("2016-05-25 13:30:00.030"),
        pd.Timestamp("2016-05-25 13:30:00.041"),
        pd.Timestamp("2016-05-25 13:30:00.048"),
        pd.Timestamp("2016-05-25 13:30:00.049"),
        pd.Timestamp("2016-05-25 13:30:00.072"),
        pd.Timestamp("2016-05-25 13:30:00.075")
    ],
    "ticker": [
       "GOOG",
       "MSFT",
       "MSFT",
       "MSFT",
       "GOOG",
       "AAPL",
       "GOOG",
       "MSFT"
   ],
   "bid": [720.50, 51.95, 51.97, 51.99, 720.50, 97.99, 720.50, 52.01],
   "ask": [720.93, 51.96, 51.98, 52.00, 720.93, 98.01, 720.88, 52.03]
})
quotes = kx.Table(data={
   "time": [
       pd.Timestamp("2016-05-25 13:30:00.023"),
       pd.Timestamp("2016-05-25 13:30:00.038"),
       pd.Timestamp("2016-05-25 13:30:00.048"),
       pd.Timestamp("2016-05-25 13:30:00.048"),
       pd.Timestamp("2016-05-25 13:30:00.048")
   ],
   "ticker": ["MSFT", "MSFT", "GOOG", "GOOG", "AAPL"],
   "price": [51.95, 51.95, 720.77, 720.92, 98.0],
   "quantity": [75, 155, 100, 100, 100]
})
trades

In [ ]:
quotes

In [ ]:
trades.merge_asof(quotes, on="time")

## Analytic functionality

In [ ]:
# All the examples in this section will use this example table.
kx.q('N: 100')
tab = kx.q('([] sym: N?`AAPL`GOOG`MSFT; price: 250f - N?500f; traded: 100 - N?200; hold: N?0b)')
tab

### Table.abs()

```
Table.abs(numeric_only=False)
```

Take the absolute value of each element in the table. This will raise an error if there are columns that contain data that have no absolute value.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| numeric_only | bool | Only use columns of the table that can be converted to an absolute value.        | False   |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Table / KeyedTable | The resulting table like object with only positive numerical values. |

In [ ]:
tab.abs(numeric_only=True)

### Table.count()

```
Table.count(axis=0, numeric_only=False)
```

Returns the count of non null values across the given axis.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to count elements across 1 is columns, 0 is rows.                | 0       |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represent the column name / row number and the values are the result of calling `count` on that column / row. |

In [ ]:
tab.count()

### Table.max()

```
Table.max(axis=0, skipna=True, numeric_only=False)
```

Returns the maximum value across the given axis.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the maximum across 0 is columns, 1 is rows.                | 0       |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represents the column name / row number and the values are the result of calling `max` on that column / row. |

In [ ]:
tab.max()

### Table.idxmin()

```
Table.idxmax(axis=0, skipna=True, numeric_only=False)
```

Return index of first occurrence of minimum over requested axis.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the idxmin across 0 is columns, 1 is rows.                | 0       |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represents the column name / row number and the values are the result of calling `idxmin` on that column / row. |

**Examples:**

Calculate the idxmin across the columns of a table

In [ ]:
tab.idxmin()

Calculate the idxmin across the rows of a table using only columns thar are of a numeric data type

In [ ]:
tab.idxmin(axis=1, numeric_only=True)

### Table.idxmax()

```
Table.idxmax(axis=0, skipna=True, numeric_only=False)
```

Return index of first occurrence of maximum over requested axis.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the idxmax across 0 is columns, 1 is rows.                | 0       |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represents the column name / row number and the values are the result of calling `idxmax` on that column / row. |

**Examples:**

Calculate the idxmax across the columns of a table

In [ ]:
tab.idxmax()

Calculate the idxmax across the rows of a table using only columns thar are of a numeric data type

In [ ]:
tab.idxmax(axis=1, numeric_only=True)

### Table.min()

```
Table.min(axis=0, skipna=True, numeric_only=False)
```

Returns the minimum value across the given axis.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the minimum across 0 is columns, 1 is rows.                | 0       |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represents the column name / row number and the values are the result of calling `min` on that column / row. |

In [ ]:
tab.min()

### Table.sum()

```
Table.sum(axis=0, skipna=True, numeric_only=False, min_count=0)
```

Returns the sum of all values across the given axis.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the sum across 0 is columns, 1 is rows.                    | 0       |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |
| min_count    | int  | If not set to 0 if there are less then `min_count` values across the axis a null value will be returned | 0 |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represents the column name / row number and the values are the result of calling `sum` on that column / row. |

In [ ]:
tab.sum()

### Table.mean()

```
Table.mean(axis=0, numeric_only=False)
```

Get the mean of values across the requested axis.

**Parameters:**

| Name         | Type | Description                                                | Default |
| :----------: | :--: | :--------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate mean across 0 is columns, 1 is rows. | 0       |
| numeric_only | bool | Include only columns / rows with numeric data.             | False   |

**Returns:**

| Type       | Description                                                                                    |
| :--------: | :--------------------------------------------------------------------------------------------- |
| Dictionary | The mean across each row / column with the key corresponding to the row number or column name. |

**Examples:**

Calculate the mean across the columns of a table

In [ ]:
tab = kx.Table(data=
    {
        'a': [1, 2, 2, 4],
        'b': [1, 2, 6, 7],
        'c': [7, 8, 9, 10],
        'd': [7, 11, 14, 14]
    }
)
tab

In [ ]:
tab.mean()

Calculate the mean across the rows of a table

In [ ]:
tab.mean(axis=1)

### Table.median()

```
Table.median(axis=0, numeric_only=False)
```

Get the median of values across the requested axis.

**Parameters:**

| Name         | Type | Description                                                  | Default |
| :----------: | :--: | :----------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate median across 0 is columns, 1 is rows. | 0       |
| numeric_only | bool | Include only columns / rows with numeric data.               | False   |

**Returns:**

| Type       | Description                                                                                      |
| :--------: | :----------------------------------------------------------------------------------------------- |
| Dictionary | The median across each row / column with the key corresponding to the row number or column name. |

**Examples:**

Calculate the median across the columns of a table

In [ ]:
tab = kx.Table(data=
    {
        'a': [1, 2, 2, 4],
        'b': [1, 2, 6, 7],
        'c': [7, 8, 9, 10],
        'd': [7, 11, 14, 14]
    }
)
tab

In [ ]:
tab.median()

Calculate the median across the rows of a table

In [ ]:
tab.median(axis=1)

### Table.mode()

```
Table.mode(axis=0, numeric_only=False, dropna=True)
```

Get the mode of values across the requested axis.

**Parameters:**

| Name         | Type | Description                                                   | Default |
| :----------: | :--: | :------------------------------------------------------------ | :-----: |
| axis         | int  | The axis to calculate mode across 0 is columns, 1 is rows.    | 0       |
| numeric_only | bool | Include only columns / rows with numeric data.                | False   |
| dropna       | bool | Remove null values from the data before calculating the mode. | True    |

**Returns:**

| Type       | Description                                                                                       |
| :--------: | :------------------------------------------------------------------------------------------------ |
| Table      | The mode across each row / column with the column corresponding to the row number or column name. |

**Examples:**

Calculate the mode across the columns of a table

In [ ]:
tab = kx.Table(data=
    {
        'a': [1, 2, 2, 4],
        'b': [1, 2, 6, 7],
        'c': [7, 8, 9, 10],
        'd': [7, 11, 14, 14]
    }
)
tab

In [ ]:
tab.mode()

Calculate the median across the rows of a table

In [ ]:
tab.mode(axis=1)

Calculate the mode across columns and keep null values.

In [ ]:
tab = kx.Table(data=
    {
        'x': [0, 1, 2, 3, 4, 5, 6, 7, np.NaN, np.NaN],
        'y': [10, 11, 12, 13, 14, 15, 16, 17, 18, np.NaN],
        'z': ['a', 'b', 'c', 'd', 'd', 'e', 'e', 'f', 'g', 'h']
    }
)
tab

In [ ]:
tab.mode(dropna=False)

### Table.prod()

```
Table.prod(axis=0, skipna=True, numeric_only=False, min_count=0)
```

Returns the product of all values across the given axis.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the product across 0 is columns, 1 is rows.                | 0       |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |
| min_count    | int  | If not set to 0 if there are less then `min_count` values across the axis a null value will be returned | 0 |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represents the column name / row number and the values are the result of calling `prd` on that column / row. |

In [ ]:
# This example will use a smaller version of the above table
# as the result of calculating the product quickly goes over the integer limits.
kx.q('N: 10')
tab = kx.q('([] sym: N?`AAPL`GOOG`MSFT; price: 2.5f - N?5f; traded: 10 - N?20; hold: N?0b)')
tab[tab['traded'] == 0, 'traded'] = 1
tab[tab['price'] == 0, 'price'] = 1.0
tab

In [ ]:
tab.prod(numeric_only=True)

### Table.kurt()

```
Table.kurt(axis=0, skipna=True, numeric_only=False)
```

Return unbiased kurtosis over requested axis. Kurtosis obtained using Fisher’s definition of kurtosis (kurtosis of normal == 0.0). Normalized by N-1.


**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | Axis for the function to be applied on. 0 is columns, 1 is rows.                    | 0       |
| skipna       | bool | not yet implemented                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |

**Returns:**

| Type       | Description                                                                              |
| :--------: | :--------------------------------------------------------------------------------------- |
| Dictionary | Map of columns and their yielded kurtosis values |

**Examples:**

Calculate the kurt across the columns of a table

In [ ]:
tab = kx.Table(data=
    {
        'a': [1, 2, 2, 4],
        'b': [1, 2, 6, 7],
        'c': [7, 8, 9, 10],
        'd': [7, 11, 14, 14]
    }
)
tab

In [ ]:
tab.kurt()

Calculate the kurtosis across the rows of a table

In [ ]:
tab.kurt(axis=1)

### Table.sem()

```
Table.sem(axis=0, skipna=True, numeric_only=False, ddof=0)
```
Return unbiased standard error of the mean over requested axis. Normalized by N-1 by default. This can be changed using the ddof argument

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the sum across 0 is columns, 1 is rows.                    | 0       |
| skipna       | bool | not yet implemented                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |
| ddof    | int  | Delta Degrees of Freedom. The divisor used in calculations is N - ddof, where N represents the number of elements. | 1 |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | The sem across each row / column with the key corresponding to the row number or column name. |

**Examples**

Calculate the sem across the columns of a table

In [ ]:
tab = kx.Table(data=
        {
            'a': [1, 2, 2, 4],
            'b': [1, 2, 6, 7],
            'c': [7, 8, 9, 10],
            'd': [7, 11, 14, 14],
        }
    )
tab

In [ ]:
tab.sem()

Calculate the sem across the rows of a table

In [ ]:
tab.sem(axis=1)

Calculate sem accross columns with ddof=0:

In [ ]:
tab.sem(ddof=0)

### Table.skew()

```
Table.skew(axis=0, skipna=True, numeric_only=False)
```

Returns the skewness of all values across the given axis.

**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the skewness across 0 is columns, 1 is rows.                | 0       |
| skipna       | bool | Ignore any null values along the axis.                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |


**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Dictionary         | A dictionary where the key represent the column name / row number and the values are the result of calling `skew` on that column / row. |

In [ ]:
tab.skew(numeric_only=True)

### Table.std()

```
Table.std(axis=0, skipna=True, numeric_only=False, ddof=0)
```

Return sample standard deviation over requested axis. Normalized by N-1 by default. This can be changed using the ddof argument.


**Parameters:**

| Name         | Type | Description                                                                      | Default |
| :----------: | :--: | :------------------------------------------------------------------------------- | :-----: |
| axis         | int  | The axis to calculate the sum across 0 is columns, 1 is rows.                    | 0       |
| skipna       | bool | not yet implemented                                           | True    |
| numeric_only | bool | Only use columns of the table that are of a numeric data type.                   | False   |
| ddof    | int  | Delta Degrees of Freedom. The divisor used in calculations is N - ddof, where N represents the number of elements. | 1 |

**Returns:**

| Type               | Description                                                          |
| :----------------: | :------------------------------------------------------------------- |
| Table         | The std across each row / column with the key corresponding to the row number or column name. |

**Examples:**

Calculate the std across the columns of a table

In [ ]:
tab = kx.Table(data=
    {
        'a': [1, 2, 2, 4],
        'b': [1, 2, 6, 7],
        'c': [7, 8, 9, 10],
        'd': [7, 11, 14, 14]
    }
)
tab

In [ ]:
tab.std()

Calculate the std across the rows of a table

In [ ]:
tab.std(axis=1)

Calculate std accross columns with ddof=0:

In [ ]:
tab.std(ddof=0)

## Group By

### Table.groupby()

```
Table.groupby(
    by=None,
    axis=0,
    level=None,
    as_index=True,
    sort=True,
    group_keys=True,
    observed=False,
    dropna=True
)
```

Group data based on like values within columns to easily apply operations on groups.

**Parameters:**

| Name             | Type | Description                                                                 | Default  |
| :--------------: | :--: | :-------------------------------------------------------------------------- | :------: |
| by         | Union[Symbol/SymbolVector/int/list] | The column name(s) or column index(es) to group the data on. | None |
| axis       | int                                 | Not Yet Implemented.                            | 0        |
| level      | Union[Symbol/SymbolVector/int/list] | The column name(s) or column index(es) to group the data on. | None | 
| as_index   | bool                                | Return the table with groups as the key column. | True     |
| sort       | bool                                | Sort the resulting table based off the key.     | True     |
| group_keys | bool                                | Not Yet Implemented.                            | True     | 
| observed   | bool                                | Not Yet Implemented.                            | False    |
| dropna     | bool                                | Drop groups where the group is null.            | True     |  

Either `by` or `level` can be used to specify the columns to group on, using both will raise an error.

Using and integer or list of integers is only possible when calling `groupby` on a `KeyedTable` object.

**Returns:**

| Type         | Description                                     |
| :----------: | :---------------------------------------------- |
| GroupbyTable | The resulting table after the grouping is done. |

**Examples:**

Example Table.

In [ ]:
tab = kx.Table(data={
    'Animal': ['Falcon', 'Falcon', 'Parrot', 'Parrot'],
    'Max Speed': [380., 370., 24., 26.],
    'Max Altitude': [570., 555., 275., 300.]
})

tab

Group on the `Animal` column and calculate the mean of the resulting `Max Speed` and `Max Altitude` columns.

In [ ]:
tab.groupby(kx.SymbolVector(['Animal'])).mean()

Example table with multiple columns to group on.

In [ ]:
tab = kx.q('2!', kx.Table(
    data={
        'Animal': ['Falcon', 'Falcon', 'Parrot', 'Parrot', 'Parrot'],
        'Type': ['Captive', 'Wild', 'Captive', 'Wild', 'Wild'],
        'Max Speed': [390., 350., 30., 20., 25.]
    }
))
tab

Group on multiple columns using thier indexes.

In [ ]:
tab.groupby(level=[0, 1]).mean()

Example table with Nulls.

In [ ]:
tab = kx.Table(
    [
        ["a", 12, 12],
        [kx.q('`'), 12.3, 33.],
        ["b", 12.3, 123],
        ["a", 1, 1]
    ],
    columns=["a", "b", "c"]
)
tab

Group on column `a` and keep null groups.

In [ ]:
tab.groupby('a', dropna=False).sum()

Group on column `a` keeping null groups and not using the groups as an index column.

In [ ]:
tab.groupby('a', dropna=False, as_index=False).sum()

## Apply

### Table.apply()

```
Table.apply(
    func,
    *args,
    axis=0,
    raw=None,
    result_type=None,
    **kwargs
)
```

Apply a function along an axis of the DataFrame.

Objects passed to a function are passed as kx list objects.

**Parameters:**

| Name             | Type                                | Description                                                                  | Default  |
| :--------------: | :---------------------------------: | :-------------------------------------------------------------------------- | :------: |
| func             | function | Function to apply to each column or row. |  |
| `*args`          | any      | Positional arguments to pass to `func` in addition to the kx list.                            | |
| axis             | int      | The axis along which the function is applied, `0` applies function to each column, `1` applied function to each row. | 0 | 
| raw              | bool     | Not yet implemented. | None     |
| result_type      | str      | Not yet implemented.     | None     |
| `**kwargs`       | dict     | Additional keyword arguments to pass as keywords to `func`, this argument is not implemented in the case `func` is a kx callable function.                            | None     | 


**Returns:**

| Type                      | Description                                     |
| :-----------------------: | :---------------------------------------------- |
| List, Dictionary or Table | Result of applying `func` along the giveen axis of the `kx.Table`. |

**Examples:**

Example Table.

In [ ]:
tab = kx.Table([[4, 9]] * 3, columns=['A', 'B'])

tab

Apply square root on each item within a column

In [ ]:
tab.apply(kx.q.sqrt)

Apply a reducing function sum on either axis

In [ ]:
tab.apply(kx.q.sum)

In [ ]:
tab.apply(lambda x: sum(x), axis=1)

## Aggregate

### Table.agg()

```
Table.agg(
    func,
    axis=0,
    *args,
    **kwargs
)
```

Aggregate data using one or more operations over a specified axis

Objects passed to a function are passed as kx vector/list objects.

**Parameters:**

| Name             | Type                                | Description                                                                  | Default  |
| :--------------: | :---------------------------------: | :-------------------------------------------------------------------------- | :------: |
| func             | function, str, list or dict | Function to use for aggregating the data. If a function this must either work when passed a `Table` or when passed to `Table.apply`<br><br>Accepted combinations are:<br><ul><li>function</li><li>string function name</li><li>list of functions and/or function names, e.g. `[kx.q.sum, 'mean']`</li><li>dict of axis labels -> functions or function names</li><ul> |  |
| `*args`          | any      | Positional arguments to pass to `func` in addition to the kx list.                            | |
| axis             | int      | The axis along which the function is applied, `0` applies function to each column, at present row based application is not supported. | 0 | 
| `**kwargs`       | dict     | Additional keyword arguments to pass as keywords to `func`, this argument is not implemented in the case `func` is a kx callable function.                            | None     | 


**Returns:**

| Type                      | Description                                     |
| :-----------------------: | :---------------------------------------------- |
| List, Dictionary or Table | Result of applying `func` along the giveen axis of the `kx.Table`. |

**Examples:**

Example Table.

In [ ]:
tab = kx.Table([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [kx.FloatAtom.null, kx.FloatAtom.null, kx.FloatAtom.null]],
                  columns=['A', 'B', 'C'])

tab

Aggregate a list of functions over rows

In [ ]:
tab.agg(['sum', 'min'])

Perform an aggregation using a user specified function

In [ ]:
import statistics
def mode(x):
    return statistics.mode(x)
tab.agg(mode)

Apply an aggregation supplying column specification for supplied function

In [ ]:
tab.agg({'A': 'max', 'B': mode})

## Data Preprocessing

### Table.add_prefix()

```
Table.add_prefix(columns)
```

Rename columns adding a prefix in a table and return the resulting Table object.

**Parameters:**

| Name    | Type            | Description                                                         | Default    |
| :-----: | :-------------: | :------------------------------------------------------------------ | :--------: |
| prefix  | str             | The string that will be concatenated with the name of the columns   | _required_ |
| axis    | int             | Axis to add prefix on.                                              | 0          |

**Returns:**

| Type  | Description                                                        |
| :---: | :----------------------------------------------------------------- |
| Table | A table with the given column(s) renamed adding a prefix.                          |

**Examples:**

The initial table to which a prefix will be added to its columns

In [ ]:
tab.head()

Add "col_" to table columns:

In [ ]:
tab.add_prefix(prefix="col_").head()

### Table.add_suffix()

```
Table.add_suffix(columns)
```

Rename columns adding a suffix in a table and return the resulting Table object.

**Parameters:**

| Name    | Type            | Description                                                         | Default    |
| :-----: | :-------------: | :------------------------------------------------------------------ | :--------: |
| suffix  | str             | The string that will be concatenated with the name of the columns   | _required_ |
| axis    | int             | Axis to add suffix on.                                              | 0          |

**Returns:**

| Type  | Description                                                        |
| :---: | :----------------------------------------------------------------- |
| Table | A table with the given column(s) renamed adding a suffix.          |

**Examples:**

The initial table to which a suffix will be added to its columns

In [ ]:
tab.head()

Add "_col" to table columns:

In [ ]:
tab.add_suffix(suffix="_col").head()

### Table.astype()

```
Table.astype(dtype, copy=True, errors='raise')
```

Cast a column/columns of the Dataframes object to a specified `dtype`.

**Parameters:**

| Name | Type | Description | Default |
| :-: | :-: | :-: | :-: |
| dtype | data type, or dict of column name -> data type | Use a PyKx wrapper data type or Python type to cast all columns to the same type. Alternatively, use {col: dtype, …}, where col is a column label and dtype is PyKx wrapper data type to cast one or more of the DataFrame’s columns to column-specific types. |  |
| copy |  Boolean | Default of True, False not implemented | True |
| errors |  {‘raise’, ‘ignore’} | If passed anything other than 'raise', it will return the dataframe | 'raise' |


**Returns:**

| Type | Description |
| :-: | :-: |
| Dataframe | The dataframe with columns casted according to passed dtypes |

**Examples:**

The examples in the section will use the example table.

In [ ]:
df = kx.q('([] c1:1 2 3i; c2:1 2 3j; c3:1 2 3h; c4:1 2 3i)')

Cast all columns to dtype LongVector

In [ ]:
df.astype(kx.LongVector)

Casting as specified in the dictionary supplied with given dtype per column

In [ ]:
df.astype({'c1':kx.LongVector, 'c2':'kx.ShortVector'})

The next example will use this table

In [ ]:
df = kx.q('([] c1:3#.z.p; c2:`abc`def`ghi; c3:1 2 3j; c4:("abc";"def";"ghi");c5:"abc";c6:(1 2 3;4 5 6;7 8 9))')

Casting char and string columns to symbol columns

In [ ]:
df.astype({'c4':kx.SymbolVector, 'c5':kx.SymbolVector})

### Table.drop()

```
Table.drop(item, axis=0)
```

Remove either columns or rows from a table and return the resulting Table object.

**Parameters:**

| Name    | Type                   | Description                                                 | Default    |
| :-----: | :--------------------: | :---------------------------------------------------------- | :--------: |
| item    | Union[str, list[str]]  | The column name(s) or row number(s) to drop from the table. | _required_ |
| axis    | int                    | The column name or list of names to pop from the table.     | 0          |

**Returns:**

| Type  | Description                                                        |
| :---: | :----------------------------------------------------------------- |
| Table | A table with the given column(s) / row(s) removed.                 |

**Examples:**

Drop rows from a table.

In [ ]:
# The examples in this section will use this example table filled with random data
kx.q('N: 1000')
tab = kx.q('([] x: til N; y: N?`AAPL`GOOG`MSFT; z: N?500f; w: N?1000; v: N?(0N 0 50 100 200 250))')
tab.head()

In [ ]:
tab.drop([0, 2, 4, 6, 8, 10]).head()

Drop columns from a table.

In [ ]:
tab.drop('y', axis=1).head()

### Table.drop_duplicates()

```
Table.drop_duplicates()
```

Remove either columns or rows from a table and return the resulting Table object.

**Returns:**

| Type  | Description                              |
| :---: | :--------------------------------------- |
| Table | A table with all duplicate rows removed. |

**Examples:**

Create a table with duplicates for the example

In [ ]:
tab2 = kx.q('([] 100?`AAPL`GOOG`MSFT; 100?3)')
tab2

Drop all duplicate rows from the table.

In [ ]:
tab2.drop_duplicates()

### Table.pop()

```
Table.pop(item)
```

Remove a column or columns from a table by column name and return the column after it has been removed.

**Parameters:**

| Name    | Type                   | Description                                             | Default    |
| :-----: | :--------------------: | :------------------------------------------------------ | :--------: |
| item    | Union[str, list[str]]  | The column name or list of names to pop from the table. | _required_ |

**Returns:**

| Type  | Description                                                        |
| :---: | :----------------------------------------------------------------- |
| Table | A table containing only the columns removed from the input table. |

**Examples:**

Remove the `v` column from the table and return it.

In [ ]:
display(tab.head())
print('\n\nPop the `v` column out of the table')
display(tab.pop("v"))
print('\n\nUpdated Table')
display(tab.head())

Remove the `z` and `w` columns from the table and return them.

In [ ]:
display(tab.head())
print('\n\nPop the `z` and `w` columns out of the table')
display(tab.pop(["z", "w"]).head())
print('\n\nUpdated Table')
display(tab.head())

### Table.rename()

```
Table.rename(labels=None, index=None, columns=None, axis=None, copy=None, inplace=False, level=None, errors='ignore')
```

Rename columns in a table and return the resulting Table object.

**Parameters:**

| Name     | Type   | Description                                                                                                     | Default |
| :------: | :----: | :------------------------------------------------------------------------------------------------------------------| :---:|
| labels   | dict   | A dictionary of either new index or column names to new names to be used in conjunction with the _axis_ parameter. | None |
| columns  | dict   | A dictionary of column name to new column name to use when renaming.                                               | None |
| index    | dict   | A dictionary of index to new index name to use when renaming keyed tables.                                         | None |
| axis     | {0 or 'index', 1 or 'columns'} | Designating the axis to be renamed by the _labels_ dictionary.                             | None |
| copy     | None   | Not yet implemented.                                                                                               | None |
| inplace  | bool   | Not yet implemented.                                                                                               | None |
| level    | None   | Not yet implemented.                                                                                               | None |
| errors   | string | Not yet implemented.                                                                                               | None |

**Returns:**

| Type  | Description                                                         |
| :---: | :-----------------------------------------------------------------  |
| Table | A table with the given columns or indices renamed.                  |

**Examples:**

The initial table we will be renaming columns on and a keyed table to rename the index on.

In [ ]:
tab.head()
key_tab = kx.KeyedTable(data=tab) 

Rename column `x` to `index` and `y` to `symbol` using the `columns` keyword.

In [ ]:
tab.rename(columns={'x': 'index', 'y': 'symbol'}).head()

Rename column `x` to `index` and `y` to `symbol` by setting the `axis` keyword.

In [ ]:
tab.rename({'x': 'index', 'y': 'symbol'}, axis = 1).head()

Rename index of a keyed table by using literal `index` as the `axis` parameter.

In [ ]:
key_tab.rename({0:"a", 1:"b"}, axis = 'index').head()

### Table.reset_index()

```
Table.reset_index(levels, *,
                  drop=False, inplace=False,
                  col_level=0, col_fill='',
                  allow_duplicates=False,
                  names=None)
```

Reset the keys/index of a keyed PyKX table. This can be used to remove/unset one or more keys within a table.

**Parameters:**

| Name | Type | Description | Default |
| :-: | :-: | :-: | :-: |
| level | int, str or list | The name/indices of the keys to be reset within the table. | None |
| drop |  Boolean | Should remaining key columns be removed from the table post index resetting. | False |
| inplace | Boolean | Not Yet Implemented | False |
| col_level | int or str | Not Yet Implemented | 0 |
| col_fill | object | Not Yet Implemented | '' |
| allow_duplicates | Boolean | Can duplicate columns be created | False |
| names | str or list | Not Yet Implemented | None |


**Returns:**

| Type | Description |
| :-: | :-: |
| Dataframe | The dataframe with table updated following index reset request |

**Examples:**

Generate data to be used for index resetting 

In [ ]:
N = 1000
qtab = kx.Table(data = {
    'x0': kx.random.random(N, ['a', 'b', 'c']),
    'x1': kx.random.random(N, ['d', 'e', 'f']),
    'x2': kx.random.random(N, ['g', 'h', 'i']),
    'y0': kx.random.random(N, 10.0),
    'y1': kx.random.random(N, 10.0),
    'y2': kx.random.random(N, kx.GUIDAtom.null)
    }).set_index(['x0', 'x1', 'x2'])
qtab

Resetting the index of the table will result in original index columns being added to the table directly

In [ ]:
qtab.reset_index()

Reset the index adding a specified named column to the table

In [ ]:
qtab.reset_index('x0')

Reset the index using multiple named columns

In [ ]:
qtab.reset_index(['x0', 'x1'])

Reset the index specifying the column `number` which is to be added to the table

In [ ]:
qtab.reset_index(0)

Reset the index specifying multiple numbered columns

In [ ]:
qtab.reset_index([0, 2])

Drop index columns from table

In [ ]:
qtab.reset_index(drop=True)

Drop specified key columns from table

In [ ]:
qtab.reset_index('x0', drop=True)

### Table.set_index()

```
Table.set_index(
    keys,
    drop=True,
    append=False,
    inplace=False,
    verify_integrity=False,
)
```

Add index/indexes to a Table/KeyedTable.

**Parameters:**

| Name             | Type | Description                                                              | Default    |
| :--------------: | :--: | :----------------------------------------------------------------------- | :--------: |
| keys             | Union[Symbol/SymbolVector/Table] | The key(s) or data to key on                 | _required_ |
| drop             | bool                             | Not Yet Implemented                          | True       |
| append           | bool                             | Whether to append columns to existing index. | False      | 
| inplace          | bool                             | Not Yet Implemented                          | False      |
| verify_integrity | bool                             | Check the new index for duplicates           | False      |    

**Returns:**

| Type       | Description                                    |
| :--------: | :--------------------------------------------- |
| KeyedTable | The resulting table after the index is applied |

**Examples:**

Adding indexes:

In [ ]:
kx.q('N: 10')
tab = kx.q('([] sym: N?`AAPL`GOOG`MSFT; price: 2.5f - N?5f; traded: N?0 1; hold: N?01b)')

In [ ]:
#Setting a single index
tab.set_index('sym')

In [ ]:
#Setting multipe indexes
tab.set_index(['sym', 'traded'])

In [ ]:
#Pass a table as index (lengths must match)
status = kx.q('{select movement from ungroup select movement:`down`up 0<=deltas price by sym from x}',tab)
tab.set_index(status)

Appending:

In [ ]:
#Default is false - previous index 'sym' deleted and replaced by 'hold'
tab.set_index('sym').set_index('hold')

In [ ]:
#append= True will retain 'sym' index and add 'hold' as second index
tab.set_index('sym').set_index('hold', append= True)

Verify Integrity:

In [ ]:
#Will allow duplicates in index:
tab.set_index('sym')

In [ ]:
#Will error as 'sym' has duplicates
try:
    tab.set_index('sym', verify_integrity= True)
except kx.QError as e:
    print(f'Caught Error: {e}')